In [1]:
import os

In [2]:
import semantic_kernel as sk
kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    )
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001F7E94F8F70>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001F7C8E19D30>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [3]:
ask = """
Tommorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in English.
Convert the text to uppercase
"""

### Providing  plugins to the plaanner

The planner needs to know what plugins are available to it. Here we'll give it access to the SummarizePlugin and WriterPlugin we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset.

You can also include native functions as well. Here we'll add the TextPlugin.

In [4]:
from semantic_kernel.core_plugins.text_plugin import TextPlugin

plugins_directory = "samples\\plugins"
summarize_plugin = kernel.import_semantic_plugin_from_directory(plugins_directory, "SummarizePlugin")
writer_plugin = kernel.import_semantic_plugin_from_directory(plugins_directory, "WriterPlugin")
text_plugin = kernel.import_plugin(TextPlugin(), "TextPlugin")

# Basic Plan

Let's start by taking a look at a basic planner. The BasicPlanner produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [6]:
basic_plan = await planner.create_plan(ask, kernel)

In [7]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterPlugin.Brainstorm"},
        {"function": "WriterPlugin.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterPlugin.Translate", "args": {"language": "English"}},
        {"function": "TextPlugin.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the plugins that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline plugin and have it be available to the Planner. Be sure to give it a function name and plugin name.

In [8]:
result = await planner.execute_plan(basic_plan, kernel)

Variable `$to` not found
Variable `$sender` not found


In [9]:
print(result)

MAKE SURE YOU ONLY USE ENGLISH.

HELLO,

I HOPE YOU ARE DOING WELL. I WANTED TO SUGGEST SOME FUN AND ROMANTIC ACTIVITIES THAT WE COULD DO TOGETHER:

1. WE COULD HAVE A PICNIC IN THE PARK, SURROUNDED BY NATURE AND FRESH AIR.
2. COOKING A ROMANTIC DINNER TOGETHER COULD BE A FUN AND INTIMATE WAY TO SPEND AN EVENING.
3. GOING TO A WINE TASTING WOULD BE A GREAT OPPORTUNITY TO TRY NEW WINES AND LEARN MORE ABOUT THEM.
4. ICE SKATING IS ALWAYS A FUN AND PLAYFUL ACTIVITY THAT WE COULD ENJOY TOGETHER.
5. TAKING A HOT AIR BALLOON RIDE WOULD BE A UNIQUE AND EXCITING EXPERIENCE THAT WE COULD SHARE.
6. WE COULD ALSO HAVE A COZY NIGHT IN AND WATCH A ROMANTIC MOVIE TOGETHER.
7. GOING ON A HIKE AND HAVING A PICNIC AT THE TOP WOULD BE A GREAT WAY TO ENJOY THE OUTDOORS AND EACH OTHER'S COMPANY.
8. TAKING A DANCE CLASS TOGETHER COULD BE A FUN AND ROMANTIC WAY TO LEARN SOMETHING NEW.
9. VISITING A LOCAL MUSEUM OR ART GALLERY WOULD BE A GREAT WAY TO APPRECIATE ART AND CULTURE TOGETHER.
10. FINALLY, WE COULD

The Plan Object Model
To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# 2. Sequential Planner

The sequentioal planner is an XML-based step-by-step planner.You can see the prompt used for it here

(https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Plugins/SequentialPlanning/skprompt.txt)

In [10]:
from semantic_kernel.planning.sequential_planner import SequentialPlanner

planner = SequentialPlanner(kernel)

In [11]:
sequential_plan = await planner.create_plan(goal=ask)

In [15]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Convert a string to uppercase. : {'variables': {'input': ''}}


In [16]:
result = await sequential_plan.invoke()

In [14]:
print(result)

# 3. Action Planner

###### The action planner takes in a list of functions and the goal, and outputs a single function to use that is appropriate to meet that goal

In [17]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

In [20]:
from semantic_kernel.core_plugins import (
    FileIOPlugin,
    MathPlugin,
    TextPlugin,
    TimePlugin
)

kernel.import_plugin(MathPlugin(), "math")
kernel.import_plugin(FileIOPlugin(), "fileIO")
kernel.import_plugin(TimePlugin(), "time")
kernel.import_plugin(TextPlugin(), "text")

KernelPlugin(name='text', description=None, functions={'lowercase': KernelFunction(plugin_name='text', description='Convert a string to lowercase.', name='lowercase', is_semantic=False, stream_function=<bound method TextPlugin.lowercase of TextPlugin()>, parameters=[], delegate_type=<DelegateTypes.InStringOutString: 9>, function=<bound method TextPlugin.lowercase of TextPlugin()>, plugins=KernelPluginCollection(plugins={'SummarizePlugin': KernelPlugin(name='SummarizePlugin', description=None, functions={'MakeAbstractReadable': KernelFunction(plugin_name='SummarizePlugin', description='Given a scientific white paper abstract, rewrite it to make it more readable', name='MakeAbstractReadable', is_semantic=True, stream_function=<function KernelFunction.from_semantic_config.<locals>._local_stream_func at 0x000001F7C8E19DC0>, parameters=[ParameterView(name='input', description='', default_value='', type_='string', required=False)], delegate_type=<DelegateTypes.ContextSwitchInKernelContextOut

In [21]:
ask = "what is the sum of 22 and 22?"

In [22]:
plan = await planner.create_plan(goal=ask)

In [23]:
result = await plan.invoke()

In [24]:
print(result)

44


# 4. Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner here.
https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu

In [25]:
from semantic_kernel.planning.stepwise_planner import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import StepwisePlannerConfig

Let's create a Bing Search native plugin that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your .env file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [29]:
class WebSearchEnginePlugin:
    """ A search engine plugin. """

    from semantic_kernel.orchestration.kernel_context import KernelContext
    from semantic_kernel.plugin_definition import kernel_function, kernel_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @kernel_function(
        description="Performs a web search for a given query",
        name = "searchAsync"
    )
    @kernel_function_context_parameter(
        name="query",
        description="The search query"
    )
    async def search(self, query: str, context: KernelContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search(query, num_results=5, offset=0)
        return str(result)

In [32]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = os.getenv("BING_API_KEY")
connector = BingConnector(BING_API_KEY)
kernel.import_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

ValueError: Bing API key cannot be null. Please set environment variable BING_API_KEY.

In [31]:
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin 

kernel.import_plugin(TimePlugin(), "timee")
kernel.import_plugin(MathPlugin(), "mathh")

KernelPlugin(name='mathh', description=None, functions={'Add': KernelFunction(plugin_name='mathh', description='Adds value to a value', name='Add', is_semantic=False, stream_function=<bound method MathPlugin.add of MathPlugin()>, parameters=[ParameterView(name='input', description='The value to add', default_value='', type_='string', required=False), ParameterView(name='Amount', description='Amount to add', default_value='', type_='number', required=True)], delegate_type=<DelegateTypes.InStringAndContextOutString: 12>, function=<bound method MathPlugin.add of MathPlugin()>, plugins=KernelPluginCollection(plugins={'SummarizePlugin': KernelPlugin(name='SummarizePlugin', description=None, functions={'MakeAbstractReadable': KernelFunction(plugin_name='SummarizePlugin', description='Given a scientific white paper abstract, rewrite it to make it more readable', name='MakeAbstractReadable', is_semantic=True, stream_function=<function KernelFunction.from_semantic_config.<locals>._local_stream_

In [33]:
planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

In [34]:
ask = """How many total championships combined do the top 5 teams in the NBA have ?"""

plan = planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke()

In [ ]:
print(result)

Let's see the steps that the AI took to get to the answer.

In [ ]:

for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))